<a href="https://colab.research.google.com/github/aileenvl/aiagentweekmty/blob/main/AIAgentWeekMTY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# L2: Create Agents to Research and Write an Article
In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determin

In [2]:
!pip install requests

In [3]:
pip install 'crewai[tools]'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: crewai-tools
    Found existing installation: crewai-tools 0.1.6
    Uninstalling crewai-tools-0.1.6:
      Successfully uninstalled crewai-tools-0.1.6


In [4]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [5]:
!pip install langchain-google-genai crewai

INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain-0.1.20-py3-none-any.whl.metadata (13 kB)
  Using cached langchain-0.1.19-py3-none-any.whl.metadata (13 kB)
  Using cached langchain-0.1.17-py3-none-any.whl.metadata (13 kB)
  Using cached langchain-0.1.16-py3-none-any.whl.metadata (13 kB)
  Using cached langchain-0.1.15-py3-none-any.whl.metadata (13 kB)
  Using cached langchain-0.1.14-py3-none-any.whl.metadata (13 kB)
INFO: pip is still looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
  Using cached embedchain-0.1.128-py3-none-any.whl.metadata (9.2 kB)
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl 

Import from the crewAI libray.

In [6]:
import os
from crewai import Agent, Task, Crew, LLM
from langchain.tools import Tool
import requests
import json

# Configurar la API key de Gemini
os.environ["GEMINI_API_KEY"] = "your-api-key"
os.environ["DEVTO_API_KEY"] = "your-api-key"

# Crear el LLM usando la forma correcta según la documentación
llm = LLM(
    model="gemini/gemini-2.0-flash-exp",  # Formato correcto con el prefijo "gemini/"
    temperature=0.7,
    api_key=os.environ["GEMINI_API_KEY"]
)


## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [7]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    llm=llm,
	verbose=True
)

### Agent: Writer

In [8]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    llm=llm,
    verbose=True
)

### Agent: Editor

In [9]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    llm=llm,
    verbose=True
)

In [10]:
from crewai.tools import BaseTool, tool
from pydantic import BaseModel, Field
from typing import Type, List, Optional

# Modelo de entrada para la herramienta de publicación en Dev.to
class DevtoPublishInput(BaseModel):
    """Esquema de entrada para publicar en Dev.to."""
    title: str = Field(..., description="Título del artículo")
    content: str = Field(..., description="Contenido en formato Markdown")
    tags: Optional[List[str]] = Field(None, description="Lista de etiquetas (máximo 4)")
    series: Optional[str] = Field(None, description="Nombre de la serie si es parte de una")
    published: bool = Field(False, description="True para publicar, False para borrador")

# Herramienta personalizada para publicar en Dev.to
class DevtoPublishTool(BaseTool):
    name: str = "publish_to_devto"
    description: str = "Publica un artículo en Dev.to usando su API"
    args_schema: Type[BaseModel] = DevtoPublishInput

    def _run(
        self,
        title: str,
        content: str,
        tags: Optional[List[str]] = None,
        series: Optional[str] = None,
        published: bool = False
    ) -> str:
        """Publica un artículo en Dev.to."""
        api_key = os.environ.get("DEVTO_API_KEY")
        if not api_key:
            return "Error: API key de Dev.to no configurada. Configura DEVTO_API_KEY."

        # Preparar los datos del artículo
        article_data = {
            "article": {
                "title": title,
                "body_markdown": content,
                "published": published
            }
        }

        # Agregar tags si se proporcionan (máximo 4 permitidos)
        if tags:
            if len(tags) > 4:
                tags = tags[:4]  # Dev.to permite máximo 4 tags
            article_data["article"]["tags"] = tags

        # Agregar serie si se proporciona
        if series:
            article_data["article"]["series"] = series

        # Enviar solicitud a la API de Dev.to
        headers = {
            "api-key": api_key,
            "Content-Type": "application/json"
        }

        try:
            response = requests.post(
                "https://dev.to/api/articles",
                json=article_data,
                headers=headers
            )

            if response.status_code not in [200, 201]:
                return f"Error al publicar: {response.status_code} - {response.text}"

            result = response.json()
            article_url = result.get('url', 'No disponible')
            return f"Artículo {'publicado' if published else 'guardado como borrador'} exitosamente. URL: {article_url}"
        except Exception as e:
            return f"Error al conectar con la API de Dev.to: {str(e)}"

# Alternativa usando el decorador @tool
@tool("Publicar en Dev.to")
def publish_to_devto_tool(
    title: str,
    content: str,
    tags: Optional[List[str]] = None,
    series: Optional[str] = None,
    published: bool = False
) -> str:
    """
    Publica un artículo en Dev.to usando su API.

    Args:
        title: Título del artículo
        content: Contenido en formato Markdown
        tags: Lista de etiquetas (máximo 4)
        series: Nombre de la serie si es parte de una
        published: True para publicar, False para borrador
    """
    api_key = os.environ.get("DEVTO_API_KEY")
    if not api_key:
        return "Error: API key de Dev.to no configurada. Configura DEVTO_API_KEY."

    # Preparar los datos del artículo
    article_data = {
        "article": {
            "title": title,
            "body_markdown": content,
            "published": published
        }
    }

    # Agregar tags si se proporcionan (máximo 4 permitidos)
    if tags and len(tags) > 0:
        if len(tags) > 4:
            tags = tags[:4]  # Dev.to permite máximo 4 tags
        article_data["article"]["tags"] = tags

    # Agregar serie si se proporciona
    if series:
        article_data["article"]["series"] = series

    # Enviar solicitud a la API de Dev.to
    headers = {
        "api-key": api_key,
        "Content-Type": "application/json"
    }

    try:
        response = requests.post(
            "https://dev.to/api/articles",
            json=article_data,
            headers=headers
        )

        if response.status_code not in [200, 201]:
            return f"Error al publicar: {response.status_code} - {response.text}"

        result = response.json()
        article_url = result.get('url', 'No disponible')
        return f"Artículo {'publicado' if published else 'guardado como borrador'} exitosamente. URL: {article_url}"
    except Exception as e:
        return f"Error al conectar con la API de Dev.to: {str(e)}"

devto_tool = DevtoPublishTool()

In [11]:
publisher = Agent(
    role="Dev.to Publisher",
    goal="Format and publish technical articles on Dev.to",
    backstory="You're a technical content strategist specializing in developer-focused content. You know how to format and optimize articles for the Dev.to platform to maximize engagement and visibility in the developer community.",
    llm=llm,
    verbose=True,
    tools=[devto_tool]
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan


In [12]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [13]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [14]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [15]:
publish = Task(
    description="""
    Prepare and publish the edited article on Dev.to. You must:
    1. Optimize the title for the developer audience
    2. Format the content properly in Markdown
    3. Select up to 4 relevant tags
    4. Decide if the article should be part of a series
    5. Save as a draft (published=False) initially

    Use the publish_to_devto tool to submit the article to Dev.to.
    """,
    expected_output="""
    A confirmation message that the article has been successfully published to Dev.to as a draft,
    including the title, tags selected, and URL of the article if available.

    The message should include:
    1. The optimized title used
    2. The tags selected (max 4)
    3. Whether it was added to a series
    4. Confirmation of successful publication as a draft
    """,
    agent=publisher,
    context=[edit]
)


Creating the Crew
Create your crew of Agents
Pass the tasks to be performed by those agents.
Note: For this simple example, the tasks will be performed sequentially (i.e they are dependent on each other), so the order of the task in the list matters.
verbose=2 allows you to see all the logs of the execution.

In [16]:
crew = Crew(
    agents=[planner, writer, editor, publisher],
    tasks=[plan, write, edit, publish],
    verbose=True
)

Running the Crew
Note: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [17]:
result = crew.kickoff(inputs={"topic": "AI agents"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d77f968c-d580-41ef-9eeb-4675232d3fbe                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 8c583739-4c54-42e9-9dbf-2842a267e1e8
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 8c583739-4c54-42e9-9dbf-2842a267e1e8
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on AI agents.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


🚀 Crew: crew
└── 📋 Task: 8c583739-4c54-42e9-9dbf-2842a267e1e8
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Dev.to Publisher
## Final Answer: 
The article "AI Agents: A Developer's Guide to Autonomous Assistants and Intelligent Automation" has been successfully published to Dev.to as a draft.

*   **Title:** AI Agents: A Developer's Guide to Autonomous Assistants and Intelligent Automation
*   **Tags:** ai, automation, agents, machinelearning
*   **Series:** Not part of a series
*   **Status:** Published as a draft
*   **URL:** https://dev.to/aileenvl/ai-agents-a-developers-guide-to-autonomous-assistants-and-intelligent-automation-32ca-temp-slug-4770313




🚀 Crew: crew
├── 📋 Task: 8c583739-4c54-42e9-9dbf-2842a267e1e8
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 8e54194a-cf36-4d83-bbc2-e3d036a8c4fd
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
├── 📋 Task: f0229f45-f4b6-49ad-b461-8bc3886f9a4b
│      Assigned to: Editor
│      Status: ✅ Completed
│   └── 🤖 Agent: Editor
│           Status: ✅ Completed
└── 📋 Task: d1ce2504-0c25-49c4-8fa5-1e7c47ec4218
       Status: Executing Task...
    └── 🤖 Agent: Dev.to Publisher
            Status: ✅ Completed
        └── 🔧 Used publish_to_devto (1)

🚀 Crew: crew
├── 📋 Task: 8c583739-4c54-42e9-9dbf-2842a267e1e8
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 8e54194a-cf36-4d83-bbc2-e3d036a8c4fd
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
├── 📋 Task: f0229f45-f4b6-49ad-b461-8bc3886f9a4b
│      Assigned to: Editor
│      Status: ✅ Completed
│   └── 🤖 Agent: Editor
│           Status: ✅ Completed
└── 📋 Task: d1ce2504-0c25-49c4-8fa5-1e7c47ec4218
       Assigned to: Dev.to Publisher
       Status: ✅ Completed
    └── 🤖 Agent: Dev.to Publisher
            Status: ✅ Completed
        └── 🔧 Used publish_to_devto (1)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d1ce2504-0c25-49c4-8fa5-1e7c47ec4218                                                                     │
│  Agent: Dev.to Publisher                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: d77f968c-d580-41ef-9eeb-4675232d3fbe                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Display the results of your execution as markdown in the notebook.

In [18]:
from IPython.display import display, Markdown

# Si result es un objeto CrewOutput, accede a su atributo 'raw'
if hasattr(result, 'raw'):
    display(Markdown(result.raw))
else:
    # Intenta extraer el contenido del markdown de la representación en string
    content = str(result)
    # Si el contenido comienza con marcadores de código markdown, elimínalos
    if content.startswith("```markdown"):
        content = content[len("```markdown"):].strip()
    if content.endswith("```"):
        content = content[:-3].strip()

    display(Markdown(content))

The article "AI Agents: A Developer's Guide to Autonomous Assistants and Intelligent Automation" has been successfully published to Dev.to as a draft.

*   **Title:** AI Agents: A Developer's Guide to Autonomous Assistants and Intelligent Automation
*   **Tags:** ai, automation, agents, machinelearning
*   **Series:** Not part of a series
*   **Status:** Published as a draft
*   **URL:** https://dev.to/aileenvl/ai-agents-a-developers-guide-to-autonomous-assistants-and-intelligent-automation-32ca-temp-slug-4770313

In [19]:
print(result)

The article "AI Agents: A Developer's Guide to Autonomous Assistants and Intelligent Automation" has been successfully published to Dev.to as a draft.

*   **Title:** AI Agents: A Developer's Guide to Autonomous Assistants and Intelligent Automation
*   **Tags:** ai, automation, agents, machinelearning
*   **Series:** Not part of a series
*   **Status:** Published as a draft
*   **URL:** https://dev.to/aileenvl/ai-agents-a-developers-guide-to-autonomous-assistants-and-intelligent-automation-32ca-temp-slug-4770313


Try it Yourself
Pass in a topic of your choice and see what the agents come up with!

In [20]:
topic = ""
result = crew.kickoff(inputs={"topic": topic})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d77f968c-d580-41ef-9eeb-4675232d3fbe                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 8c583739-4c54-42e9-9dbf-2842a267e1e8
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 8c583739-4c54-42e9-9dbf-2842a267e1e8
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on .
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


🚀 Crew: crew
└── 📋 Task: 8c583739-4c54-42e9-9dbf-2842a267e1e8
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Dev.to Publisher
## Final Answer: 
The article "Sustainable Packaging: A Developer's Guide to Eco-Friendly Choices" has been successfully published to Dev.to as a draft. The tags selected are: sustainability, packaging, environment, and innovation. It was not added to a series. The URL of the draft article is: https://dev.to/aileenvl/sustainable-packaging-a-developers-guide-to-eco-friendly-choices-4072-temp-slug-13195149




🚀 Crew: crew
├── 📋 Task: 8c583739-4c54-42e9-9dbf-2842a267e1e8
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 8e54194a-cf36-4d83-bbc2-e3d036a8c4fd
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
├── 📋 Task: f0229f45-f4b6-49ad-b461-8bc3886f9a4b
│      Assigned to: Editor
│      Status: ✅ Completed
│   └── 🤖 Agent: Editor
│           Status: ✅ Completed
└── 📋 Task: d1ce2504-0c25-49c4-8fa5-1e7c47ec4218
       Status: Executing Task...
    └── 🤖 Agent: Dev.to Publisher
            Status: ✅ Completed
        └── 🔧 Used publish_to_devto (2)

🚀 Crew: crew
├── 📋 Task: 8c583739-4c54-42e9-9dbf-2842a267e1e8
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 8e54194a-cf36-4d83-bbc2-e3d036a8c4fd
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
├── 📋 Task: f0229f45-f4b6-49ad-b461-8bc3886f9a4b
│      Assigned to: Editor
│      Status: ✅ Completed
│   └── 🤖 Agent: Editor
│           Status: ✅ Completed
└── 📋 Task: d1ce2504-0c25-49c4-8fa5-1e7c47ec4218
       Assigned to: Dev.to Publisher
       Status: ✅ Completed
    └── 🤖 Agent: Dev.to Publisher
            Status: ✅ Completed
        └── 🔧 Used publish_to_devto (2)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d1ce2504-0c25-49c4-8fa5-1e7c47ec4218                                                                     │
│  Agent: Dev.to Publisher                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: d77f968c-d580-41ef-9eeb-4675232d3fbe                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯